In [139]:
using Test

In [140]:
f = readlines("day7.input");

In [141]:
function parse_line(line)
    if startswith(line, "\$ cd")
        return ("cd", split(line, ' ')[3])
    end
    if startswith(line, "\$ ls")
        return ("ls", nothing)
    end
    if startswith(line, "dir")
        return ("dir", split(line, ' ')[2])
    end
    
    return ("file", parse(Int64, split(line, ' ')[1]))
end

parse_line (generic function with 1 method)

In [142]:
mutable struct Directory
    children::Vector{Directory}
    size::Int64
    parent::Union{Directory, Nothing}
    name::String
end

In [143]:
function get_size(dir::Directory)
    child_total = 0
    if length(dir.children) == 0
        return dir.size
    end
    for child in dir.children
        child_total += get_size(child)
    end
    return child_total + dir.size
end

get_size (generic function with 1 method)

In [144]:
function Directory(name, parent)
    Directory(Vector{Directory}(), 0, parent, name)
end

Directory

In [145]:
function has_subdir(dir1::Directory, dir_name::AbstractString)
    for dir in dir1.children
        if dir_name == dir.name
            return true
        end
    end

    return false
end

has_subdir (generic function with 1 method)

In [146]:
function change_directory(operand::AbstractString, root::Directory, current_dir::Directory)
    if operand == ".."
        return current_dir.parent
    elseif operand == "/"
        return root
    else
        if (has_subdir(current_dir, operand))
            for child in current_dir.children
                if child.name == operand
                    return child
                    break
                end
            end
        else
            new_dir = Directory(operand, current_dir)
            push!(current_dir.children, new_dir)
            return new_dir
        end
    end
end

change_directory (generic function with 2 methods)

In [147]:
function process_file(current_dir, file_size)
    current_dir.size += file_size
end

process_file (generic function with 1 method)

In [148]:
function get_score(dir, max_size)
    score = 0
    size = get_size(dir)
    if size < max_size
        score += size
    end
    for child in dir.children
        score += get_score(child, max_size)
    end

    score
end

get_score (generic function with 1 method)

In [149]:
function solve_part_1(input, max_size)
    root = Directory("/", nothing)
    current_dir = root

    for line in input
        operator, operand = parse_line(line)
        if operator == "cd"
            current_dir = change_directory(operand, root, current_dir)
        elseif operator == "file"
            process_file(current_dir, operand)
        end
    end

    get_score(root, max_size)
end


solve_part_1 (generic function with 1 method)

In [150]:
@test solve_part_1(String.(split("\$ cd /
\$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
\$ cd a
\$ ls
dir e
29116 f
2557 g
62596 h.lst
\$ cd e
\$ ls
584 i
\$ cd ..
\$ cd ..
\$ cd d
\$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k", "\n")), 100000) == 95437


Test Passed

In [151]:
solve_part_1(f, 100000)

1453349

In [152]:
function get_smallest_dir_bigger_than(dir, size_requirement)
    smallest = 70000000

    for child in dir.children
        child_size = get_smallest_dir_bigger_than(child, size_requirement)
        smallest = min(smallest, child_size)
    end

    size = get_size(dir)
    if size >= size_requirement
        smallest = min(smallest, size)
    end

    smallest
end

get_smallest_dir_bigger_than (generic function with 1 method)

In [153]:
function solve_part_2(input)
    root = Directory("/", nothing)
    current_dir = root

    for line in input
        operator, operand = parse_line(line)
        if operator == "cd"
            current_dir = change_directory(operand, root, current_dir)
        elseif operator == "file"
            process_file(current_dir, operand)
        end
    end

    root_size = get_size(root)
    required_size = 30000000 - (70000000 -root_size)
    get_smallest_dir_bigger_than(root, required_size)
end


solve_part_2 (generic function with 1 method)

In [154]:
@test solve_part_2(String.(split("\$ cd /
\$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
\$ cd a
\$ ls
dir e
29116 f
2557 g
62596 h.lst
\$ cd e
\$ ls
584 i
\$ cd ..
\$ cd ..
\$ cd d
\$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k", "\n"))) == 24933642


Test Passed

In [155]:
solve_part_2(f)

2948823